In [8]:
######################################################################################
# Using Electrodermal Activity to Detect Deception and Suspicion during a Card Game
# Affective Computing - Mini-project
# Jan Ondras
# Dec 2017 - Jan 2018
######################################################################################
######################################################################################
# Statistics/distributions about epoch (deception/suspicion) leghts
######################################################################################


# TRUTH EPOCH LENGTH (without thresholding)
mean_dec_len = {}

import glob
import csv
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Iterate over games (10 in total)
game_events_files = glob.glob('./../Experiment/AnnotatedEvents/G*.csv')
for game_events_file_name in game_events_files:
      
    # Extract subject IDs of players
    P1_SID = game_events_file_name[-8:-6]
    P2_SID = game_events_file_name[-6:-4]
    print "Extracting subjects: ", P1_SID, P2_SID
    
    P1_cnt = P2_cnt = 0
    P1_lengths = []
    P2_lengths = []
    
    events_data = np.loadtxt(game_events_file_name, delimiter=",", skiprows=1, dtype=str)
    for i, event in enumerate(events_data):
        
        event_type = event[1]
        event_P1_label = event[2]
        event_P2_label = event[3]        
        
        if event_type == 'EOG':
            pass
        else:

            # Check and correct (if needed) timestamp format
            if len(event[0]) == 8 and event[0][2] == ':' and event[0][5] == ':':
                events_data[i][0] = event[0] + '.000' # system time as string
            elif len(event[0]) == 12 and event[0][2] == ':' and event[0][5] == ':' and event[0][8] == '.':
                pass
            else:
                print "Timestamp error ", event
        
            if event_type == 'S':
                pass
            elif event_type == 'D' and i > 0 and events_data[i-1][1] == 'D':

                epoch_start = datetime.strptime(events_data[i-1][0], '%H:%M:%S.%f')
                epoch_end = datetime.strptime(events_data[i][0], '%H:%M:%S.%f')
                epoch_length = (epoch_end-epoch_start).total_seconds()  # in seconds

                if event_P1_label in ['D','H','S','C'] and event_P2_label == '': # P1 discarded
                    if event_P1_label == event[4]: # lie 
                        P1_lengths.append(epoch_length)
                        P1_cnt += 1
                elif event_P2_label in ['D','H','S','C'] and event_P1_label == '': # P2 discarded
                    if event_P2_label == event[4]: # lie 
                        P2_lengths.append(epoch_length)
                        P2_cnt += 1
                else:
                    print "Event S ", event, " not labeled"

            else:
                if i > 0 and events_data[i-1][1] not in ['S','EOG']:
                    print "Event type error in: ", game_events_file_name, "; ", event
    
    mean_dec_len[P1_SID] = np.mean(P1_lengths)
    mean_dec_len[P2_SID] = np.mean(P2_lengths)
    print P1_SID, P1_cnt, "", P2_SID, P2_cnt
    print P1_lengths
    print P2_lengths
    print np.mean(P1_lengths), np.min(P1_lengths), np.max(P1_lengths)
    print np.mean(P2_lengths), np.min(P2_lengths), np.max(P2_lengths)
#     print P1_cnt + P2_cnt

print "\nDictionary"
print mean_dec_len
print "mean=", np.mean(mean_dec_len.values()), np.std(mean_dec_len.values())

Extracting subjects:  11 12
11 52  12 61
[5.0, 3.0, 4.0, 2.0, 4.0, 2.0, 1.0, 1.0, 1.0, 3.0, 1.0, 2.5, 1.0, 0.5, 4.0, 2.0, 5.0, 1.0, 1.0, 3.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 6.0, 6.0, 4.0, 4.0, 2.0, 0.5, 1.0, 1.0, 3.0, 1.0, 3.0, 1.0, 3.0, 1.0, 2.0, 6.0, 2.0, 1.0, 1.0, 2.0, 0.5, 1.0, 18.0, 1.0, 2.0, 1.0]
[5.0, 7.0, 4.0, 4.0, 3.0, 1.0, 1.0, 2.0, 2.0, 1.0, 0.5, 1.0, 1.0, 4.0, 9.5, 11.0, 8.5, 5.0, 5.0, 2.0, 3.0, 3.0, 3.0, 4.0, 4.0, 1.0, 7.0, 3.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 3.0, 3.0, 1.5, 2.0, 2.0, 1.0, 0.5, 0.5, 1.0, 4.0, 1.0, 6.0, 2.0, 5.0, 4.0, 2.0, 2.0, 0.5, 0.5, 6.0, 2.0, 3.0, 5.5, 0.5, 1.0, 4.0, 9.0]
2.43269230769 0.5 18.0
3.06557377049 0.5 11.0
Extracting subjects:  09 10
09 16  10 23
[10.0, 6.0, 7.0, 2.0, 5.0, 7.0, 9.0, 10.0, 3.0, 1.0, 4.0, 5.0, 1.0, 2.0, 2.0, 3.0]
[2.0, 2.0, 5.0, 3.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 3.0, 4.0, 4.0, 1.0, 0.5, 2.0]
4.8125 1.0 10.0
1.82608695652 0.5 5.0
Extracting subjects:  17 18
17 37  18 32
[1.0, 3.0, 8.0, 7.0, 21.0

In [2]:
# DECEPTION EPOCH LENGTH (without thresholding)
mean_dec_len = {}

import glob
import csv
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Iterate over games (10 in total)
game_events_files = glob.glob('./../Experiment/AnnotatedEvents/G*.csv')
for game_events_file_name in game_events_files:
      
    # Extract subject IDs of players
    P1_SID = game_events_file_name[-8:-6]
    P2_SID = game_events_file_name[-6:-4]
    print "Extracting subjects: ", P1_SID, P2_SID
    
    P1_cnt = P2_cnt = 0
    P1_lengths = []
    P2_lengths = []
    
    events_data = np.loadtxt(game_events_file_name, delimiter=",", skiprows=1, dtype=str)
    for i, event in enumerate(events_data):
        
        event_type = event[1]
        event_P1_label = event[2]
        event_P2_label = event[3]        
        
        if event_type == 'EOG':
            pass
        else:

            # Check and correct (if needed) timestamp format
            if len(event[0]) == 8 and event[0][2] == ':' and event[0][5] == ':':
                events_data[i][0] = event[0] + '.000' # system time as string
            elif len(event[0]) == 12 and event[0][2] == ':' and event[0][5] == ':' and event[0][8] == '.':
                pass
            else:
                print "Timestamp error ", event
        
            if event_type == 'S':
                pass
            elif event_type == 'D' and i > 0 and events_data[i-1][1] == 'D':

                epoch_start = datetime.strptime(events_data[i-1][0], '%H:%M:%S.%f')
                epoch_end = datetime.strptime(events_data[i][0], '%H:%M:%S.%f')
                epoch_length = (epoch_end-epoch_start).total_seconds()  # in seconds

                if event_P1_label in ['D','H','S','C'] and event_P2_label == '': # P1 discarded
                    if event_P1_label != event[4]: # lie 
                        P1_lengths.append(epoch_length)
                        P1_cnt += 1
                elif event_P2_label in ['D','H','S','C'] and event_P1_label == '': # P2 discarded
                    if event_P2_label != event[4]: # lie 
                        P2_lengths.append(epoch_length)
                        P2_cnt += 1
                else:
                    print "Event S ", event, " not labeled"

            else:
                if i > 0 and events_data[i-1][1] not in ['S','EOG']:
                    print "Event type error in: ", game_events_file_name, "; ", event
    
    mean_dec_len[P1_SID] = np.mean(P1_lengths)
    mean_dec_len[P2_SID] = np.mean(P2_lengths)
    print P1_SID, P1_cnt, "", P2_SID, P2_cnt
    print P1_lengths
    print P2_lengths
    print np.mean(P1_lengths), np.min(P1_lengths), np.max(P1_lengths)
    print np.mean(P2_lengths), np.min(P2_lengths), np.max(P2_lengths)
#     print P1_cnt + P2_cnt

print "\nDictionary"
print mean_dec_len
print "mean=", np.mean(mean_dec_len.values()), np.std(mean_dec_len.values())

Extracting subjects:  11 12
11 42  12 31
[2.0, 4.0, 3.0, 2.5, 3.0, 1.0, 6.0, 3.0, 8.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 2.0, 3.0, 1.0, 1.0, 0.5, 3.0, 2.5, 2.0, 1.0, 1.0, 4.0, 3.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 3.0, 1.5, 3.0, 1.0, 2.0, 16.0]
[11.0, 13.0, 4.0, 4.0, 2.0, 0.5, 5.0, 1.0, 6.0, 4.0, 2.0, 1.0, 3.0, 1.5, 1.0, 1.0, 3.0, 0.5, 2.0, 2.0, 3.0, 4.0, 1.0, 5.0, 2.0, 1.0, 1.0, 2.0, 5.0, 3.0, 3.0]
2.29761904762 0.5 16.0
3.14516129032 0.5 13.0
Extracting subjects:  09 10
09 37  10 34
[9.0, 7.0, 5.0, 5.0, 4.0, 4.0, 4.0, 2.0, 4.0, 6.0, 6.0, 1.0, 6.0, 2.0, 2.0, 2.0, 1.0, 2.0, 1.0, 3.0, 2.0, 1.0, 1.0, 2.0, 3.0, 3.0, 3.0, 0.5, 0.5, 2.0, 5.0, 5.0, 12.0, 3.0, 3.0, 1.0, 0.5]
[5.0, 2.0, 6.0, 4.0, 4.0, 2.0, 2.0, 1.0, 10.0, 1.0, 1.0, 2.0, 1.0, 4.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, 1.0, 0.5, 1.0, 1.0, 5.0, 6.0, 2.0, 2.0, 6.0, 2.0, 2.0, 2.0]
3.33783783784 0.5 12.0
2.57352941176 0.5 10.0
Extracting subjects:  17 18
17 12  18 30
[2.0, 15.0, 9.0, 2.0, 3.0, 3.0, 2.0, 6.0, 12.0, 5.0, 6.0

In [3]:
# Calculate mean of both DECEPTION AND TRUTH epoch lengths for each subject (without thresholding)
mean_dec_len = {}

import glob
import csv
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Iterate over games (10 in total)
game_events_files = glob.glob('./../Experiment/AnnotatedEvents/G*.csv')
for game_events_file_name in game_events_files:
      
    # Extract subject IDs of players
    P1_SID = game_events_file_name[-8:-6]
    P2_SID = game_events_file_name[-6:-4]
    print "Extracting subjects: ", P1_SID, P2_SID
    
    P1_cnt = P2_cnt = 0
    P1_lengths = []
    P2_lengths = []
    
    events_data = np.loadtxt(game_events_file_name, delimiter=",", skiprows=1, dtype=str)
    for i, event in enumerate(events_data):
        
        event_type = event[1]
        event_P1_label = event[2]
        event_P2_label = event[3]        
        
        if event_type == 'EOG':
            pass
        else:

            # Check and correct (if needed) timestamp format
            if len(event[0]) == 8 and event[0][2] == ':' and event[0][5] == ':':
                events_data[i][0] = event[0] + '.000' # system time as string
            elif len(event[0]) == 12 and event[0][2] == ':' and event[0][5] == ':' and event[0][8] == '.':
                pass
            else:
                print "Timestamp error ", event
        
            if event_type == 'S':
                pass
            elif event_type == 'D' and i > 0 and events_data[i-1][1] == 'D':

                epoch_start = datetime.strptime(events_data[i-1][0], '%H:%M:%S.%f')
                epoch_end = datetime.strptime(events_data[i][0], '%H:%M:%S.%f')
                epoch_length = (epoch_end-epoch_start).total_seconds()  # in seconds

                if event_P1_label in ['D','H','S','C'] and event_P2_label == '': # P1 discarded
                    P1_lengths.append(epoch_length)
                    P1_cnt += 1
                elif event_P2_label in ['D','H','S','C'] and event_P1_label == '': # P2 discarded
                    P2_lengths.append(epoch_length)
                    P2_cnt += 1
                else:
                    print "Event S ", event, " not labeled"

            else:
                if i > 0 and events_data[i-1][1] not in ['S','EOG']:
                    print "Event type error in: ", game_events_file_name, "; ", event
    
    mean_dec_len[P1_SID] = np.mean(P1_lengths)
    mean_dec_len[P2_SID] = np.mean(P2_lengths)
    print P1_SID, P1_cnt, "", P2_SID, P2_cnt
    print P1_lengths
    print P2_lengths
    print np.mean(P1_lengths), np.min(P1_lengths), np.max(P1_lengths)
    print np.mean(P2_lengths), np.min(P2_lengths), np.max(P2_lengths)
#     print P1_cnt + P2_cnt

print "\nDictionary"
print mean_dec_len
print "mean=", np.mean(mean_dec_len.values()), np.std(mean_dec_len.values())

Extracting subjects:  11 12
11 94  12 92
[5.0, 2.0, 3.0, 4.0, 4.0, 2.0, 4.0, 2.0, 1.0, 1.0, 1.0, 3.0, 1.0, 3.0, 2.5, 2.5, 3.0, 1.0, 6.0, 3.0, 8.0, 0.5, 1.0, 0.5, 4.0, 1.0, 2.0, 5.0, 1.0, 1.0, 3.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 6.0, 1.0, 6.0, 2.0, 4.0, 4.0, 2.0, 0.5, 1.0, 1.0, 3.0, 3.0, 1.0, 1.0, 0.5, 1.0, 3.0, 2.5, 3.0, 1.0, 2.0, 1.0, 3.0, 1.0, 4.0, 3.0, 1.0, 2.0, 6.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 2.0, 0.5, 1.0, 3.0, 1.5, 18.0, 3.0, 1.0, 2.0, 1.0, 16.0, 2.0, 1.0]
[11.0, 5.0, 7.0, 13.0, 4.0, 4.0, 4.0, 4.0, 3.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 0.5, 0.5, 1.0, 5.0, 1.0, 1.0, 4.0, 6.0, 9.5, 11.0, 8.5, 4.0, 2.0, 5.0, 1.0, 5.0, 3.0, 2.0, 3.0, 3.0, 3.0, 4.0, 1.5, 4.0, 1.0, 1.0, 1.0, 7.0, 3.0, 3.0, 0.5, 1.0, 1.0, 2.0, 2.0, 2.0, 2.0, 1.0, 3.0, 3.0, 1.5, 2.0, 3.0, 2.0, 2.0, 4.0, 1.0, 0.5, 1.0, 0.5, 1.0, 4.0, 5.0, 2.0, 1.0, 1.0, 1.0, 6.0, 2.0, 5.0, 4.0, 2.0, 2.0, 2.0, 0.5, 0.5, 5.0, 6.0, 2.0, 3.0, 3.0, 5.5, 0.5, 1.0, 4.0, 9.0, 3.0]
2.37234042553 0.5 18.0

In [4]:
# Calculate mean SUSPICION epoch lengths for each subject (without thresholding)
mean_susp_len = {}

import glob
import csv
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Iterate over games (10 in total)
game_events_files = glob.glob('./../Experiment/AnnotatedEvents/G*.csv')
for game_events_file_name in game_events_files:
      
    # Extract subject IDs of players
    P1_SID = game_events_file_name[-8:-6]
    P2_SID = game_events_file_name[-6:-4]
    print "Extracting subjects: ", P1_SID, P2_SID
    
    P1_cnt = P2_cnt = 0
    P1_lengths = []
    P2_lengths = []
    
    events_data = np.loadtxt(game_events_file_name, delimiter=",", skiprows=1, dtype=str)
    for i, event in enumerate(events_data):
        
        event_type = event[1]
        event_P1_label = event[2]
        event_P2_label = event[3]        
        
        if event_type == 'EOG':
            pass
        else:

            # Check and correct (if needed) timestamp format
            if len(event[0]) == 8 and event[0][2] == ':' and event[0][5] == ':':
                events_data[i][0] = event[0] + '.000' # system time as string
            elif len(event[0]) == 12 and event[0][2] == ':' and event[0][5] == ':' and event[0][8] == '.':
                pass
            else:
                print "Timestamp error ", event
        
            if event_type == 'D':
                pass
            elif event_type == 'S':

                epoch_start = datetime.strptime(events_data[i-1][0], '%H:%M:%S.%f')
                epoch_end = datetime.strptime(events_data[i][0], '%H:%M:%S.%f')
                epoch_length = (epoch_end-epoch_start).total_seconds()  # in seconds

                if event_P1_label == 'CH': # P1 said cheat
                    P1_lengths.append(epoch_length)
                    P1_cnt += 1
                elif event_P2_label == 'CH': # P2 said cheat
                    P2_lengths.append(epoch_length)
                    P2_cnt += 1
                else:
                    print "Event S ", event, " not labeled"

            else:
                print "Event type error in: ", game_events_file_name, "; ", event
    
    mean_susp_len[P1_SID] = np.mean(P1_lengths)
    mean_susp_len[P2_SID] = np.mean(P2_lengths)
    print P1_SID, P1_cnt, "", P2_SID, P2_cnt
    print P1_lengths
    print P2_lengths
    print np.mean(P1_lengths), np.min(P1_lengths), np.max(P1_lengths)
    print np.mean(P2_lengths), np.min(P2_lengths), np.max(P2_lengths)
#     print P1_cnt + P2_cnt

print "\nDictionary"
print mean_susp_len
print "mean=", np.mean(mean_susp_len.values()), np.std(mean_susp_len.values())

Extracting subjects:  11 12
11 32  12 32
[3.0, 4.0, 0.5, 0.5, 2.0, 0.5, 0.5, 3.0, 1.0, 1.0, 10.0, 1.0, 1.0, 0.5, 4.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 3.0, 1.0, 1.0, 2.0, 2.0, 1.0, 0.5, 2.5, 3.0]
[3.0, 1.0, 1.0, 3.0, 2.0, 2.0, 2.0, 1.0, 3.0, 2.0, 1.0, 2.0, 3.0, 5.0, 3.0, 1.0, 3.0, 1.0, 4.0, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 5.0, 19.0, 8.0, 1.0]
1.8125 0.5 10.0
2.59375 0.5 19.0
Extracting subjects:  09 10
09 7  10 3
[3.0, 11.0, 2.0, 2.0, 0.5, 2.0, 4.0]
[6.0, 2.0, 1.0]
3.5 0.5 11.0
3.0 1.0 6.0
Extracting subjects:  17 18
17 30  18 7
[2.0, 2.0, 6.0, 3.0, 9.0, 1.0, 5.0, 2.0, 1.0, 2.0, 2.0, 0.5, 4.0, 1.0, 2.0, 1.0, 10.0, 7.0, 2.0, 1.0, 0.5, 1.0, 6.0, 2.0, 1.0, 1.0, 12.0, 8.0, 1.0, 7.0]
[1.0, 2.0, 9.0, 3.0, 6.0, 2.0, 2.0]
3.43333333333 0.5 12.0
3.57142857143 1.0 9.0
Extracting subjects:  07 08
07 46  08 32
[1.0, 9.0, 1.0, 2.0, 1.0, 1.0, 11.0, 1.0, 1.0, 4.0, 1.0, 1.0, 2.0, 4.0, 2.0, 3.0, 4.0, 1.0, 2.0, 2.0, 2.0, 3.0, 1.5, 4.0, 1.0, 1.0, 2.0, 5.0, 2.0, 1.0, 2.0, 2.0, 1.0, 0.5

In [5]:
# Create dictionary of mean SUSPICION epoch lengths for each subject
# ignoring epochs with length > threshold (started to talk / ...)
# THRESHOLDED !
mean_susp_len = {}

import glob
import csv
from scipy.signal import butter, filtfilt
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

thr = 5.

# Iterate over games (10 in total)
game_events_files = glob.glob('./../Experiment/AnnotatedEvents/G*.csv')
for game_events_file_name in game_events_files:
      
    # Extract subject IDs of players
    P1_SID = game_events_file_name[-8:-6]
    P2_SID = game_events_file_name[-6:-4]
    print "Extracting subjects: ", P1_SID, P2_SID
    
    P1_cnt = P2_cnt = 0
#     mean_susp_len[P1_SID] = mean_susp_len[P2_SID] = 0.
    P1_lengths = []
    P2_lengths = []
    
    events_data = np.loadtxt(game_events_file_name, delimiter=",", skiprows=1, dtype=str)
    for i, event in enumerate(events_data):
        
        event_type = event[1]
        event_P1_label = event[2]
        event_P2_label = event[3]        
        
        if event_type == 'EOG':
            pass
        else:

            # Check and correct (if needed) timestamp format
            if len(event[0]) == 8 and event[0][2] == ':' and event[0][5] == ':':
                events_data[i][0] = event[0] + '.000' # system time as string
            elif len(event[0]) == 12 and event[0][2] == ':' and event[0][5] == ':' and event[0][8] == '.':
                pass
            else:
                print "Timestamp error ", event
        
            if event_type == 'D':
                pass
            elif event_type == 'S':

                epoch_start = datetime.strptime(events_data[i-1][0], '%H:%M:%S.%f')
                epoch_end = datetime.strptime(events_data[i][0], '%H:%M:%S.%f')
                epoch_length = (epoch_end-epoch_start).total_seconds()  # in seconds

                if event_P1_label == 'CH': # P1 said cheat
#                     mean_susp_len[P1_SID] += epoch_length
                    if epoch_length < thr:
                        P1_lengths.append(epoch_length)
                        P1_cnt += 1
                elif event_P2_label == 'CH': # P2 said cheat
#                     mean_susp_len[P2_SID] += epoch_length
                    if epoch_length < thr:
                        P2_lengths.append(epoch_length)
                        P2_cnt += 1
                else:
                    print "Event S ", event, " not labeled"

            else:
                print "Event type error in: ", game_events_file_name, "; ", event
    
    mean_susp_len[P1_SID] = np.mean(P1_lengths)
    mean_susp_len[P2_SID] = np.mean(P2_lengths)
    print P1_SID, P1_cnt, "", P2_SID, P2_cnt
    print P1_lengths
    print P2_lengths
    print np.mean(P1_lengths), np.min(P1_lengths), np.max(P1_lengths)
    print np.mean(P2_lengths), np.min(P2_lengths), np.max(P2_lengths)
#     print P1_cnt + P2_cnt

print "\nDictionary"
print mean_susp_len
print "mean=", np.mean(mean_susp_len.values()), np.std(mean_susp_len.values())

Extracting subjects:  11 12
11 31  12 28
[3.0, 4.0, 0.5, 0.5, 2.0, 0.5, 0.5, 3.0, 1.0, 1.0, 1.0, 1.0, 0.5, 4.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 3.0, 1.0, 1.0, 2.0, 2.0, 1.0, 0.5, 2.5, 3.0]
[3.0, 1.0, 1.0, 3.0, 2.0, 2.0, 2.0, 1.0, 3.0, 2.0, 1.0, 2.0, 3.0, 3.0, 1.0, 3.0, 1.0, 4.0, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 0.5, 1.0, 1.0]
1.54838709677 0.5 4.0
1.64285714286 0.5 4.0
Extracting subjects:  09 10
09 6  10 2
[3.0, 2.0, 2.0, 0.5, 2.0, 4.0]
[2.0, 1.0]
2.25 0.5 4.0
1.5 1.0 2.0
Extracting subjects:  17 18
17 21  18 5
[2.0, 2.0, 3.0, 1.0, 2.0, 1.0, 2.0, 2.0, 0.5, 4.0, 1.0, 2.0, 1.0, 2.0, 1.0, 0.5, 1.0, 2.0, 1.0, 1.0, 1.0]
[1.0, 2.0, 3.0, 2.0, 2.0]
1.57142857143 0.5 4.0
2.0 1.0 3.0
Extracting subjects:  07 08
07 43  08 27
[1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 2.0, 4.0, 2.0, 3.0, 4.0, 1.0, 2.0, 2.0, 2.0, 3.0, 1.5, 4.0, 1.0, 1.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 0.5, 4.0, 0.5, 3.0, 2.0, 1.0, 3.0, 2.0, 1.0, 1.0, 0.5, 0.5, 2.0]
[3.0, 2.0, 1.0, 1.0, 3.0, 1.0, 0.5, 1.0, 1.0, 1.0